In [ ]:
#Step 1 Preprocessing

import os
import cv2
import matplotlib.pyplot as plt
!pip install ultralytics
from ultralytics import YOLO

# Define paths
binary_detector_path = "/content/binary defect v3.pt"     # binary defective/non-defective YOLOv8 model
classifier_path = "/content/fault_classifier 11.pt"          # classifier model (multi-class)
input_image_path = "/content/images (1).jpeg"                #full pane;

# Prepare directories
os.makedirs("cell_crops", exist_ok=True)

# Load and preprocess the image
original = cv2.imread(input_image_path)
if original is None:
    raise ValueError("Image not found at provided path!")

gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
cv2.imwrite("gray_input.jpg", gray)  



In [ ]:
#Step 2 : Defective Model

# Detect defective cells using binary detector
print("🔍 Detecting *defective* cells using binary model...")
binary_model = YOLO(binary_detector_path)
binary_results = binary_model("gray_input.jpg")[0]


In [ ]:
#Step 3: resizing defective models
print("✂️  Cropping detected defective cells...")
cell_crops = []
coords = []

for i, box in enumerate(binary_results.boxes.xyxy):
    cls = int(binary_results.boxes.cls[i].item())
    label = binary_results.names[cls]

    # Only keep 'Defective' cells (if 1 = defective, 0 = non-defective)
    if "defect" in label.lower() or cls == 1:
        x1, y1, x2, y2 = map(int, box)
        crop = gray[y1:y2, x1:x2]
        resized_crop = cv2.resize(crop, (224, 224))
        crop_path = f"cell_crops/cell_{i}.jpg"
        cv2.imwrite(crop_path, resized_crop)
        cell_crops.append(crop_path)
        coords.append((x1, y1, x2, y2))
    else:
        continue  # Skip non-defective cells


In [ ]:
#Step 4 : Classifier Model
# Classify each defective cell with the 11-class classifier
print("🧠 Classifying defective cells...")
classifier = YOLO(classifier_path)
predictions = []

for crop_path in cell_crops:
    pred_result = classifier.predict(crop_path, imgsz=224, verbose=False)[0]
    class_name = pred_result.names[pred_result.probs.top1]
    confidence = pred_result.probs.top1conf
    predictions.append((class_name, confidence))
 

In [ ]:
#Step 5: Annotating and saving outputs

# Annotate the image with classification results
print("🖍️ Annotating predictions on image...")
annotated = original.copy()
for (x1, y1, x2, y2), (label, conf) in zip(coords, predictions):
    color = (0, 255, 0) if "no" in label.lower() else (0, 0, 255)
    cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
    cv2.putText(annotated, f"{label} ({conf:.2f})", (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

# Save and display results
output_path = "annotated_output.jpg"
cv2.imwrite(output_path, annotated)

plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Defective Cells with Fault Type")
plt.show()

print("✅ Pipeline complete! Final output saved to:", output_path)